In [1]:
"""Tutorial: CEPA0 and CCD"""

__author__    = "Adam S. Abbott"
__credit__    = ["Adam S. Abbott", "Justin M. Turney"]

__copyright__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2017-05-23"

# Introduction
In this tutorial, we will implement the coupled-electron pair approximation (CEPA0) and coupled-cluster doubles (CCD) methods using our spin orbital framework covered in the [previous tutorial](8a_Intro_to_spin_orbital_postHF.ipynb).


### I. Coupled Cluster Theory

In single reference coupled cluster theory, dynamic correlation is acquired by operating an exponential operator on some reference determinant, such as a Hartree-Fock wavefunction, to obtain the coupled cluster wavefunction given by:

\begin{equation}
\mid \mathrm{\Psi_{CC}} \rangle = \exp(\hat{T}) \mid \mathrm{\Phi} \rangle 
\end{equation}

where $\hat{T} = T_1 + T_2 + ... + T_n$ is the sum of "cluster operators" which act on our reference wavefunction to excite electrons from occupied ($i, j, k$...) to virtual ($a, b, c$...) orbitals. In second quantization, these cluster operators are expressed as:

\begin{equation}
T_k = \left(\frac{1}{k!}\right)^2 \sum_{\substack{i_1 \ldots i_k \\ a_1 \ldots a_k }} t_{i_1 \ldots i_k}^{a_1 \ldots a_k}   a_{a_1}^{\dagger} \ldots a_{a_k}^{\dagger} a_{i_k} \ldots a_{i_1}
\end{equation}

where $t$ is the $t$-amplitude, and $a^{\dagger}$ and $a$ are creation and annihilation operators.

### II. Coupled Cluster Doubles
For CCD, we only include the doubles cluster operator:

\begin{equation}
\mid \mathrm{\Psi_{CCD}} \rangle = \exp(T_2) \mid \mathrm{\Phi} \rangle
\end{equation}

The CCD Schr&ouml;dinger equation is

\begin{equation}
\hat{H} \mid \mathrm{\Psi_{CCD}} \rangle = E \mid \mathrm{\Psi_{CCD}}\rangle
\end{equation}

The details will not be covered here, but if we project the CCD Schr&ouml;dinger equation on the left by our Hartree-Fock reference determinant $ \langle \mathrm{\Phi}\mid $, assuming intermediate normalization $\langle \Phi \mid \mathrm{\Psi_{CCD}} \rangle = 1$, we obtain:

\begin{equation}
 \langle \Phi \mid \hat{H} \space \exp(T_2) \mid \Phi \rangle = E
\end{equation}

which is most easily evaluated with a diagrammatic application of Wick's theorem. Assuming Brillouin's theorem applies (that is, our reference is a Hartree-Fock wavefunction) we obtain:

\begin{equation}
E_{\mathrm{CCD}} = \tfrac{1}{4} \bar{g}_{ij}^{ab} t_{ab}^{ij}
\end{equation}

A somewhat more involved derivation is that of the $t$-amplitudes. These are obtained in a similar fashion to the energy expression, this time projecting the CCD Schr&ouml;dinger equation on the left by a doubly-excited reference determinant $ \langle\Phi_{ij}^{ab}\mid $:

\begin{equation}
\langle\Phi_{ij}^{ab}\mid \hat{H} \space \exp(T_2) \mid \Phi \rangle
\end{equation}

I will spare you the details of solving this expectation value as well. But, if one evaluates the diagrams via Wick's theorem and simplifies, the $t$-amplitudes are given by:

\begin{equation}
t_{ab}^{ij} = (\mathcal{E}_{ab}^{ij})^{-1} \left( \bar{g}_{ab}^{ij} + \tfrac{1}{2} \bar{g}_{ab}^{cd} t_{cd}^{ij} + \tfrac{1}{2} \bar{g}_{kl}^{ij} t_{ab}^{kl}  + \hat{P}_{(a \space / \space b)}^{(i \space / \space j)} \bar{g}_{ak}^{ic} t_{bc}^{jk} - \tfrac{1}{2}\hat{P}_{(a \space / \space b)} \bar{g}_{kl}^{cd} t_{ac}^{ij} t_{bd}^{kl} - \tfrac{1}{2}  \hat{P}^{(i \space / \space j)} \bar{g}_{kl}^{cd} t_{ab}^{ik} t_{cd}^{jl} + \tfrac{1}{4} \bar{g}_{kl}^{cd} t_{cd}^{ij} t_{ab}^{kl} +  \hat{P}^{(i \space / \space j)} \bar{g}_{kl}^{cd} t_{ac}^{ik} t_{bd}^{jl} \right)
\end{equation}

where $(\mathcal{E}_{ab}^{ij})^{-1}$ is the orbital energy denominator, more familiarly known as

\begin{equation}
(\mathcal{E}_{ab}^{ij})^{-1} = \frac{1}{\epsilon_i + \epsilon_j - \epsilon_a - \epsilon_b}
\end{equation}

and $\bar{g}_{pq}^{rs}$ is the antisymmetrized two-electron integral in physicist's notation $\langle pq \mid\mid rs \rangle$. $\hat{P}$ is the *antisymmetric permutation operator*. This operator acts on a term to produce the sum of the permutations of the indicated indices, with an appropriate sign factor. Its effect is best illustrated by an example. Consider the fourth term, which is really four terms in one. 

$\hat{P}_{(a \space / \space b)}^{(i \space / \space j)} \bar{g}_{ak}^{ic} t_{bc}^{jk}$ produces: 

1. The original: $ \quad \bar{g}_{ak}^{ic} t_{bc}^{jk} \\ $

2. Permuation of $a$ and $b$: $ \quad  \textrm{-} \bar{g}_{bk}^{ic} t_{ac}^{jk} \\ $

3. Permuation of $i$ and $j$: $ \quad \, \, \textrm{-} \bar{g}_{ak}^{jc} t_{bc}^{ik} \\ $

4. Permuation of $a$ and $b$, $i$ and $j$: $ \quad \bar{g}_{bk}^{jc} t_{ac}^{ik} \\ $


Note that each permutation adds a sign change. This shorthand notation keeps the equation in a more manageable form. 

Since the $t$-amplitudes and the energy depend on $t$-amplitudes, we must iteratively solve these equations until they reach self consistency, and the energy converges to some threshold.

### III. Retrieving MP2 and CEPA0 from the CCD equations
It is interesting to note that if we only consider the first term of the expression for the doubles amplitude $t_{ab}^{ij}$ and plug it into the energy expression, we obtain the MP2 energy expression:

\begin{equation}
t_{ab}^{ij} = (\mathcal{E}_{ab}^{ij})^{-1} \bar{g}_{ab}^{ij} 
\end{equation}

\begin{equation}
E_{\mathrm{MP2}} = \tfrac{1}{4}  \bar{g}_{ij}^{ab} t_{ab}^{ij}  = \tfrac{1}{4} \bar{g}_{ij}^{ab} \bar{g}_{ab}^{ij}   (\mathcal{E}_{ab}^{ij})^{-1}
\end{equation}

Furthermore, if we leave out the quadratic terms in the CCD amplitude equation (terms containing two $t$-amplitudes), we obtain the coupled electron-pair approximation (CEPA0):
\begin{equation}
t_{ab}^{ij} = (\mathcal{E}_{ab}^{ij})^{-1} \left( \bar{g}_{ab}^{ij} + \tfrac{1}{2} \bar{g}_{ab}^{cd} t_{cd}^{ij} + \tfrac{1}{2} \bar{g}_{kl}^{ij} t_{ab}^{kl}  + \hat{P}_{(a \space / \space b)}^{(i \space / \space j)} \bar{g}_{ak}^{ic} t_{bc}^{jk} \right)
\end{equation}

The CEPA0 energy expression is identical:

\begin{equation}
E_{\mathrm{CEPA0}} = \tfrac{1}{4} \bar{g}_{ij}^{ab} t_{ab}^{ij}
\end{equation}

Using our spin orbital setup for the MO coefficients, orbital energies, and two-electron integrals used in the [previous tutorial](8a_Intro_to_spin_orbital_postHF.ipynb), we are equipped to program the expressions for the CEPA0 and CCD correlation energy.

### Implementation: CEPA0 and CCD
As usual, we import Psi4 and NumPy, and set the appropriate options. 

In [1]:
# ==> Import statements & Global Options <==
import psi4
import numpy as np

psi4.set_memory(int(2e9))
numpy_memory = 2
psi4.core.set_output_file('output.dat', False)

In [2]:
# ==> Molecule & Psi4 Options Definitions <==
mol = psi4.geometry("""
0 1
O
H 1 1.1
H 1 1.1 2 104
symmetry c1
""")



psi4.set_options({'basis':        '6-31g',
                  'scf_type':     'pk',
                  'reference':    'rhf',
                  'mp2_type':     'conv',
                  'e_convergence': 1e-8,
                  'd_convergence': 1e-8})

Note that since we are using a spin orbital setup, we are free to use any Hartree-Fock reference we want. Here we choose RHF. For convenience, we let Psi4 take care of the Hartree-Fock procedure, and return the wavefunction object.

In [3]:
# Get the SCF wavefunction & energies
scf_e, scf_wfn = psi4.energy('scf', return_wfn=True)

Load in information about the basis set and orbitals using MintsHelper and the wavefunction:

In [4]:
mints = psi4.core.MintsHelper(scf_wfn.basisset())
nbf = mints.nbf()           # number of basis functions
nso = 2 * nbf               # number of spin orbitals
nalpha = scf_wfn.nalpha()   # number of alpha electrons
nbeta = scf_wfn.nbeta()     # number of beta electrons
nocc = nalpha + nbeta       # number of occupied orbitals
nvirt = 2 * nbf - nocc      # number of virtual orbitals

Spin-block our MO coefficients and two-electron integrals, just like in the spin orbital MP2 code:

In [5]:
Ca = np.asarray(scf_wfn.Ca())
Cb = np.asarray(scf_wfn.Cb())
C = np.block([
             [      Ca           ,   np.zeros_like(Cb) ],
             [np.zeros_like(Ca)  ,          Cb         ]
            ])

# Result: | Ca  0 |
#         | 0   Cb|


In [7]:
# Get the two electron integrals using MintsHelper
I = np.asarray(mints.ao_eri())

def spin_block_tei(I):
    """  
    Function that spin blocks two-electron integrals
    Using np.kron, we project I into the space of the 2x2 identity, tranpose the result
    and project into the space of the 2x2 identity again. This doubles the size of each axis.
    The result is our two electron integral tensor in the spin orbital form.
    """
    identity = np.eye(2)
    I = np.kron(identity, I)
    return np.kron(identity, I.T)

# Spin-block the two electron integral array
I_spinblock = spin_block_tei(I)


Convert two-electron integrals to antisymmetrized physicist's notation:

In [8]:
# Converts chemist's notation to physicist's notation, and antisymmetrize
# (pq | rs) ---> <pr | qs>
# Physicist's notation
tmp = I_spinblock.transpose(0, 2, 1, 3)
# Antisymmetrize:
# <pr||qs> = <pr | qs> - <pr | sq>
gao = tmp - tmp.transpose(0, 1, 3, 2)


Obtain the orbital energies, append them, and sort the columns of our MO coefficient matrix according to the increasing order of orbital energies. 

In [9]:
# Get orbital energies 
eps_a = np.asarray(scf_wfn.epsilon_a())
eps_b = np.asarray(scf_wfn.epsilon_b())
eps = np.append(eps_a, eps_b)

# Before sorting the orbital energies, we can use their current arrangement to sort the columns
# of C. Currently, each element i of eps corresponds to the column i of C, but we want both
# eps and columns of C to be in increasing order of orbital energies

# Sort the columns of C according to the order of increasing orbital energies 
C = C[:, eps.argsort()] 

# Sort orbital energies in increasing order
eps = np.sort(eps) 


Finally, we transform our two-electron integrals to the MO basis. Here, we denote the integrals as `gmo` to differentiate from the chemist's notation integrals `I_mo`.

In [10]:
# Transform gao, which is the spin-blocked 4d array of physicist's notation, 
# antisymmetric two-electron integrals, into the MO basis using MO coefficients 
gmo = np.einsum('pQRS, pP -> PQRS',
      np.einsum('pqRS, qQ -> pQRS',
      np.einsum('pqrS, rR -> pqRS',
      np.einsum('pqrs, sS -> pqrS', gao, C), C), C), C)


Construct the 4-dimensional array of orbital energy denominators:

In [11]:
# Define slices, create 4 dimensional orbital energy denominator tensor
n = np.newaxis
o = slice(None, nocc)
v = slice(nocc, None)
e_abij = 1 / (-eps[v, n, n, n] - eps[n, v, n, n] + eps[n, n, o, n] + eps[n, n, n, o])

We now have everything we need to construct our $t$-amplitudes and iteratively solve for our CEPA0 and CCD energy. To build the $t$-amplitudes, we first construct an empty 4-dimensional array to store them. 

In [12]:
# Create space to store t amplitudes
t_amp = np.zeros((nvirt, nvirt, nocc, nocc))


# Implementation: CEPA0
First we will program CEPA0. Recall the expression for the $t$-amplitudes:

\begin{equation}
t_{ab}^{ij} = (\mathcal{E}_{ab}^{ij})^{-1} \left( \bar{g}_{ab}^{ij} + \tfrac{1}{2} \bar{g}_{ab}^{cd} t_{cd}^{ij} + \tfrac{1}{2} \bar{g}_{kl}^{ij} t_{ab}^{kl}  + \hat{P}_{(a \space / \space b)}^{(i \space / \space j)} \bar{g}_{ak}^{ic} t_{bc}^{jk} \right)
\end{equation}

These terms translate naturally into code using NumPy's `einsum` function. To access only the occupied and virtual indices of `gmo` we use our slices defined above. The permutation operator terms can be easily obtained by transposing the original result accordingly. To construct each iteration's $t$-amplitude:  

~~~python
mp2    = gmo[v, v, o, o]
cepa1  = (1 / 2) * np.einsum('abcd, cdij -> abij', gmo[v, v, v, v], t_amp)
cepa2  = (1 / 2) * np.einsum('klij, abkl -> abij', gmo[o, o, o, o], t_amp)
cepa3a = np.einsum('akic, bcjk -> abij', gmo[v, o, o, v], t_amp)
cepa3b = -cepa3a.transpose(1, 0, 2, 3)
cepa3c = -cepa3a.transpose(0, 1, 3, 2)
cepa3d =  cepa3a.transpose(1, 0, 3, 2)
cepa3  =  cepa3a + cepa3b + cepa3c + cepa3d

t_amp_new = e_abij * (mp2 + cepa1 + cepa2 + cepa3)
~~~

To evaluate the energy, $E_{\mathrm{CEPA0}} = \tfrac{1}{4} \bar{g}_{ij}^{ab} t_{ab}^{ij}$,

~~~python
E_CEPA0 = (1 / 4) * np.einsum('ijab, abij ->', gmo[o, o, v, v], t_amp_new)
~~~

Putting it all together, we initialize the energy, set the max iterations, and iterate the energy until it converges to our convergence criterion:

In [13]:
# Initialize energy
E_CEPA0 = 0.0

MAXITER = 50

for cc_iter in range(MAXITER + 1):
    E_old = E_CEPA0
    
    # Collect terms
    mp2    = gmo[v, v, o, o]
    cepa1  = (1 / 2) * np.einsum('abcd, cdij -> abij', gmo[v, v, v, v], t_amp)
    cepa2  = (1 / 2) * np.einsum('klij, abkl -> abij', gmo[o, o, o, o], t_amp)
    cepa3a = np.einsum('akic, bcjk -> abij', gmo[v, o, o, v], t_amp)
    cepa3b = -cepa3a.transpose(1, 0, 2, 3)
    cepa3c = -cepa3a.transpose(0, 1, 3, 2)
    cepa3d =  cepa3a.transpose(1, 0, 3, 2)
    cepa3  =  cepa3a + cepa3b + cepa3c + cepa3d

    # Update t amplitude
    t_amp_new = e_abij * (mp2 + cepa1 + cepa2 + cepa3)

    # Evaluate Energy
    E_CEPA0 = (1 / 4) * np.einsum('ijab, abij ->', gmo[o, o, v, v], t_amp_new)
    t_amp = t_amp_new
    dE = E_CEPA0 - E_old
    print('CEPA0 Iteration %3d: Energy = %4.12f dE = %1.5E' % (cc_iter, E_CEPA0, dE))

    if abs(dE) < 1.e-8:
        print("\nCEPA0 Iterations have converged!")
        break

    if (cc_iter == MAXITER):
        psi4.core.clean()
        raise Exception("\nMaximum number of iterations exceeded.")

print('\nCEPA0 Correlation Energy: %5.15f' % (E_CEPA0))
print('CEPA0 Total Energy: %5.15f' % (E_CEPA0 + scf_e))

CEPA0 Iteration   0: Energy = -0.142119840437 dE = -1.42120E-01
CEPA0 Iteration   1: Energy = -0.142244391415 dE = -1.24551E-04
CEPA0 Iteration   2: Energy = -0.146403556138 dE = -4.15916E-03
CEPA0 Iteration   3: Energy = -0.147737945027 dE = -1.33439E-03
CEPA0 Iteration   4: Energy = -0.148357998825 dE = -6.20054E-04
CEPA0 Iteration   5: Energy = -0.148640319609 dE = -2.82321E-04
CEPA0 Iteration   6: Energy = -0.148774677817 dE = -1.34358E-04
CEPA0 Iteration   7: Energy = -0.148840007532 dE = -6.53297E-05
CEPA0 Iteration   8: Energy = -0.148872388226 dE = -3.23807E-05
CEPA0 Iteration   9: Energy = -0.148888687704 dE = -1.62995E-05
CEPA0 Iteration  10: Energy = -0.148897003705 dE = -8.31600E-06
CEPA0 Iteration  11: Energy = -0.148901298109 dE = -4.29440E-06
CEPA0 Iteration  12: Energy = -0.148903540584 dE = -2.24248E-06
CEPA0 Iteration  13: Energy = -0.148904723847 dE = -1.18326E-06
CEPA0 Iteration  14: Energy = -0.148905354385 dE = -6.30537E-07
CEPA0 Iteration  15: Energy = -0.1489056

Since `t_amp` is initialized to zero, the very first iteration should be the MP2 correlation energy. We can check the final CEPA0 energy with Psi4. The method is called `lccd`, or linear CCD, since CEPA0 omits the terms with two cluster amplitudes.

In [14]:
psi4.compare_values(psi4.energy('lccd'), E_CEPA0 + scf_e, 6, 'CEPA0 Energy')

	CEPA0 Energy......................................................PASSED


True

# Implementation: CCD

To code CCD, we only have to add in the last four terms in our expression for the $t$-amplitudes: 

\begin{equation}
t_{ab}^{ij} = (\mathcal{E}_{ab}^{ij})^{-1} \left( \bar{g}_{ab}^{ij} + \tfrac{1}{2} \bar{g}_{ab}^{cd} t_{cd}^{ij} + \tfrac{1}{2} \bar{g}_{kl}^{ij} t_{ab}^{kl}  + \hat{P}_{(a \space / \space b)}^{(i \space / \space j)} \bar{g}_{ak}^{ic} t_{bc}^{jk} - \underline{\tfrac{1}{2}\hat{P}_{(a \space / \space b)} \bar{g}_{kl}^{cd} t_{ac}^{ij} t_{bd}^{kl} - \tfrac{1}{2}  \hat{P}^{(i \space / \space j)} \bar{g}_{kl}^{cd} t_{ab}^{ik} t_{cd}^{jl} + \tfrac{1}{4} \bar{g}_{kl}^{cd} t_{cd}^{ij} t_{ab}^{kl} +  \hat{P}^{(i \space / \space j)} \bar{g}_{kl}^{cd} t_{ac}^{ik} t_{bd}^{jl}} \right)
\end{equation}

which we readily translate into `einsum`'s:

~~~python
ccd1a  = np.einsum('klcd, acij, bdkl -> abij', gmo[o, o, v, v], t_amp, t_amp)
ccd1b  = -ccd1a.transpose(1, 0, 2, 3)
ccd1   = -(1 / 2) * (ccd1a + ccd1b)

ccd2a  = np.einsum('klcd, abik, cdjl -> abij', gmo[o, o, v, v], t_amp, t_amp)
ccd2b  = -ccd2a.transpose(0, 1, 3, 2)
ccd2   = -(1 / 2) * (ccd2a + ccd2b)

ccd3   =  (1 / 4) * np.einsum('klcd, cdij, abkl -> abij', gmo[o, o, v, v], t_amp, t_amp)

ccd4a  = np.einsum('klcd, acik, bdjl -> abij', gmo[o, o, v, v], t_amp, t_amp)
ccd4b  = -ccd4a.transpose(0, 1, 3, 2)
ccd4   = (ccd4a + ccd4b)
~~~

and the energy expression is identical to CEPA0:
\begin{equation}
E_{CCD } = \tfrac{1}{4} \bar{g}_{ij}^{ab} t_{ab}^{ij}
\end{equation}

Adding the above terms to our CEPA0 code will compute the CCD correlation energy (may take a minute or two to run):

In [38]:
# Create space to store t amplitudes
t_amp = np.zeros((nvirt, nvirt, nocc, nocc))

# Initialize energy
E_CCD = 0.0

for cc_iter in range(1, MAXITER + 1):
    E_old = E_CCD

    # Collect terms
    mp2    = gmo[v, v, o, o]
    cepa1  = (1 / 2) * np.einsum('abcd, cdij -> abij', gmo[v, v, v, v], t_amp)
    cepa2  = (1 / 2) * np.einsum('klij, abkl -> abij', gmo[o, o, o, o], t_amp)
    cepa3a = np.einsum('akic, bcjk -> abij', gmo[v, o, o, v], t_amp)
    cepa3b = -cepa3a.transpose(1, 0, 2, 3)
    cepa3c = -cepa3a.transpose(0, 1, 3, 2)
    cepa3d =  cepa3a.transpose(1, 0, 3, 2)
    cepa3  =  cepa3a + cepa3b + cepa3c + cepa3d

    ccd1a_ref  = np.einsum('klcd, acij, bdkl -> abij', gmo[o, o, v, v], t_amp, t_amp)
    ccd1a_tmp = np.einsum('klcd,bdkl->cb', gmo[o, o, v, v], t_amp)
    ccd1a = np.einsum("cb,acij->abij", ccd1a_tmp, t_amp)
    print(np.allclose(ccd1a_ref, ccd1a))
    
    ccd1b  = -ccd1a.transpose(1, 0, 2, 3)
    ccd1   = -(1 / 2) * (ccd1a + ccd1b)

    ccd2a_ref  = np.einsum('klcd, abik, cdjl -> abij', gmo[o, o, v, v], t_amp, t_amp)
    ccd2a_tmp = np.einsum('klcd,cdjl->jk', gmo[o, o, v, v], t_amp)
    ccd2a = np.einsum("jk,abik->abij", ccd2a_tmp, t_amp)
    print(np.allclose(ccd2a_ref, ccd2a))
    
    ccd2b  = -ccd2a.transpose(0, 1, 3, 2)
    ccd2   = -(1 / 2) * (ccd2a + ccd2b)

    ccd3_ref   =  (1 / 4) * np.einsum('klcd, cdij, abkl -> abij', gmo[o, o, v, v], t_amp, t_amp)
    ccd3_tmp = np.einsum("klcd,cdij->klij", gmo[o, o, v, v], t_amp)
    ccd3 = (1 / 4) * np.einsum("klij,abkl->abij", ccd3_tmp, t_amp)
    print(np.allclose(ccd3_ref, ccd3))

    ccd4a_ref  = np.einsum('klcd, acik, bdjl -> abij', gmo[o, o, v, v], t_amp, t_amp)
    ccd4a_tmp = np.einsum("klcd,acik->laid", gmo[o, o, v, v], t_amp)
    ccd4a = np.einsum("laid,bdjl->abij", ccd4a_tmp, t_amp)
    print(np.allclose(ccd4a_ref, ccd4a))
    
    ccd4b  = -ccd4a.transpose(0, 1, 3, 2)
    ccd4   = (ccd4a + ccd4b)

    # Update Amplitude
    t_amp_new = e_abij * (mp2 + cepa1 + cepa2 + cepa3 + ccd1 + ccd2 + ccd3 + ccd4)

    # Evaluate Energy
    E_CCD = (1 / 4) * np.einsum('ijab, abij ->', gmo[o, o, v, v], t_amp_new)
    t_amp = t_amp_new
    dE = E_CCD - E_old
    print('CCD Iteration %3d: Energy = %4.12f dE = %1.5E' % (cc_iter, E_CCD, dE))

    if abs(dE) < 1.e-8:
        print("\nCCD Iterations have converged!")
        break

    if (cc_iter == MAXITER):
        psi4.core.clean()
        raise Exception("\nMaximum number of iterations exceeded.")

print('\nCCD Correlation Energy:    %15.12f' % (E_CCD))
print('CCD Total Energy:         %15.12f' % (E_CCD + scf_e))


True
True
True
True
CCD Iteration   1: Energy = -0.142119840437 dE = -1.42120E-01
True
True
True
True
CCD Iteration   2: Energy = -0.142920458247 dE = -8.00618E-04
True
True
True
True
CCD Iteration   3: Energy = -0.146174466624 dE = -3.25401E-03
True
True
True
True
CCD Iteration   4: Energy = -0.147222337375 dE = -1.04787E-03
True
True
True
True
CCD Iteration   5: Energy = -0.147660208148 dE = -4.37871E-04
True
True
True
True
CCD Iteration   6: Energy = -0.147845023190 dE = -1.84815E-04
True
True
True
True
CCD Iteration   7: Energy = -0.147926013863 dE = -8.09907E-05
True
True
True
True
CCD Iteration   8: Energy = -0.147962311822 dE = -3.62980E-05
True
True
True
True
CCD Iteration   9: Energy = -0.147978892349 dE = -1.65805E-05
True
True
True
True
CCD Iteration  10: Energy = -0.147986584357 dE = -7.69201E-06
True
True
True
True
CCD Iteration  11: Energy = -0.147990201081 dE = -3.61672E-06
True
True
True
True
CCD Iteration  12: Energy = -0.147991921970 dE = -1.72089E-06
True
True
True
T

Unfortunately, Psi4 does not have a CCD code to compare this to. However, Psi4 does have Bruekner CCD, an orbital-optimized variant of CCD. We can qualitatively compare our energies to this energy. The Bruekner-CCD energy should be a little lower than our CCD energy due to the orbital optimization procedure.

In [40]:
psi4_bccd = psi4.energy('bccd', ref_wfn = scf_wfn)
print('\nPsi4 BCCD Correlation Energy:    %15.12f' % (psi4_bccd - scf_e))
print('Psi4 BCCD Total Energy:         %15.12f' % psi4_bccd)


Psi4 BCCD Correlation Energy:    -0.149207662865
Psi4 BCCD Total Energy:         -76.101736707606


## References

1. Modern review of coupled-cluster theory, included diagrammatic derivations of the CCD equations:
	> [[Bartlett and Musial:2007](https://journals.aps.org/rmp/abstract/10.1103/RevModPhys.79.291)] Rodney J. Bartlett and Monika Musial,  "Coupled-cluster theory in quantum chemistry" *Rev. Mod. Phys.* **79**, 291 (2007)
   
2. Background on CEPA:
    >Kutzelnigg, Werner 1977 *Methods of Electronic Structure Theory* ed. H. F. Schaefer III (Plenum, New York), p 129

3. More CEPA:
    > [Koch and Kutzelnigg:1981](https://link.springer.com/article/10.1007/BF00553396) S. Koch and W. Kutzelnigg, *Theor. Chim. Acta* **59**, 387 (1981). 

4. Original CCD Paper:
    > [Čížek:1966](http://aip.scitation.org/doi/abs/10.1063/1.1727484) Jiří Čížek, "On the Correlation Problem in Atomic and Molecular Systems. Calculation of Wavefunction Components in Ursell‐Type Expansion Using Quantum‐Field Theoretical Methods" *J. Chem. Phys* **45**, 4256 (1966)  

5. Useful notes on diagrams applied to post-HF methods:
    > A. V. Copan, "Diagram notation" accessed with https://github.com/CCQC/chem-8950/tree/master/2017
